# MLP para Predição de Indicadores Sociais

### Importação das bibliotecas

In [ ]:
import pandas as pd
import numpy as np
import xlrd
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import tqdm

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


### Extração dos Arquivos

In [38]:

arquivo = xlrd.open_workbook('datasets/Tabelas1a21.xls')

planilhas_alvo = ['tab2', 'tab4', 'tab6', 'tab8', 'tab19']
abas = {}

for nome in planilhas_alvo:
    abas[nome] = arquivo.sheet_by_name(nome)
    
capitais = [
    'Porto Velho', 'Rio Branco', 'Manaus', 'Boa Vista', 'Belém', 'Macapá', 'Palmas',
    'São Luís', 'Teresina', 'Fortaleza', 'Natal', 'João Pessoa', 'Recife', 'Maceió', 
    'Aracaju', 'Salvador', 'Belo Horizonte', 'Vitória', 'Rio de Janeiro', 'São Paulo',
    'Curitiba', 'Florianópolis', 'Porto Alegre', 'Campo Grande', 'Cuiabá', 'Goiânia', 'Brasília'
]

capitais_treinamento = capitais[:20]
capitais_teste = capitais[20:]

print(f"Capitais para treino ({len(capitais_treinamento)}): {capitais_treinamento}")
print(f"Capitais para teste ({len(capitais_teste)}): {capitais_teste}")


Capitais para treino (20): ['Porto Velho', 'Rio Branco', 'Manaus', 'Boa Vista', 'Belém', 'Macapá', 'Palmas', 'São Luís', 'Teresina', 'Fortaleza', 'Natal', 'João Pessoa', 'Recife', 'Maceió', 'Aracaju', 'Salvador', 'Belo Horizonte', 'Vitória', 'Rio de Janeiro', 'São Paulo']
Capitais para teste (7): ['Curitiba', 'Florianópolis', 'Porto Alegre', 'Campo Grande', 'Cuiabá', 'Goiânia', 'Brasília']


Tabela 2 (Domicílios por sexo - 2000)

In [39]:
def extrair_tabela2():
    aba2 = abas['tab2']
    dados = {}
    
    # achar ano 2000
    start_linha = None
    for i in range(aba2.nrows):
        if '2000' in str(aba2.cell_value(i, 0)):
            start_linha = i + 1
            break
    
    if start_linha is None:
        raise ValueError("Não encontrou dados de 2000 na Tabela 2")
    
    for i in range(start_linha, aba2.nrows):
        capital = str(aba2.cell_value(i, 0)).strip()
        if capital in capitais_treinamento:
            perc_mulheres = aba2.cell_value(i, 5)
            dados[capital] = perc_mulheres
    
    return dados

Tabela 4 (Mulheres +15 anos responsáveis por domicílios - 2000)

In [40]:
def extrair_tabela4():
    aba4 = abas['tab4']
    dados = {}

    start_linha = None
    for i in range(aba4.nrows):
        if '2000' in str(aba4.cell_value(i, 0)):
            start_linha = i + 1
            break
    
    if start_linha is None:
        raise ValueError("Não encontrou dados de 2000 na Tabela 4")
    
    for i in range(start_linha, aba4.nrows):
        capital = str(aba4.cell_value(i, 0)).strip()
        if capital in capitais_treinamento:
            perc_25_29 = aba4.cell_value(i, 4)
            perc_35_39 = aba4.cell_value(i, 6)
            perc_45_49 = aba4.cell_value(i, 8)
            dados[capital] = {
                'perc_25_29': perc_25_29,
                'perc_35_39': perc_35_39,
                'perc_45_49': perc_45_49
            }
    
    return dados

Tabela 6 (Responsáveis por domicílios, por média de anos de estudo e sexo - 2000)

In [41]:
def extrair_tabela6():
    aba6 = abas['tab6']
    dados = {}
    
    start_linha = None
    for i in range(aba6.nrows):
        if '2000' in str(aba6.cell_value(i, 0)):
            start_linha = i + 1
            break
    
    if start_linha is None:
        raise ValueError("Não encontrou dados de 2000 na Tabela 6")
    
    for i in range(start_linha, aba6.nrows):
        capital = str(aba6.cell_value(i, 0)).strip() 
        if capital in capitais_treinamento:
            anos_estudo = aba6.cell_value(i, 1)
            dados[capital] = anos_estudo
    
    return dados

Tabela 8 (Rendimento mensal médio, mulheres 10+ anos - 2000)

In [42]:
def extrair_tabela8():
    aba8 = abas['tab8']
    dados = {}
    
    start_linha = None
    for i in range(aba8.nrows):
        if '2000' in str(aba8.cell_value(i, 0)):
            start_linha = i + 1
            break
    
    if start_linha is None:
        raise ValueError("Não encontrou dados de 2000 na Tabela 8")
    
    for i in range(start_linha, aba8.nrows):
        capital = str(aba8.cell_value(i, 0)).strip()
        if capital in capitais_treinamento:
            rendimento = aba8.cell_value(i, 2)
            dados[capital] = rendimento
    
    return dados

Tabela 19 (Proporção população 10+ anos por alfabetização e sexo - 2000)

In [43]:
def extrair_tabela19():
    aba19 = abas['tab19']
    dados = {}
    
    start_linha = None
    for i in range(aba19.nrows):
        if '2000' in str(aba19.cell_value(i, 0)):
            start_linha = i + 1
            break
    
    if start_linha is None:
        raise ValueError("Não encontrou dados de 2000 na Tabela 19")
    
    for i in range(start_linha, aba19.nrows):
        capital = str(aba19.cell_value(i, 0)).strip()
        if capital in capitais_treinamento:
            try:
                perc_criancas = aba19.cell_value(i, 3)
                dados[capital] = perc_criancas
            except:
                for col in range(1, aba19.ncols):
                    try:
                        valor = aba19.cell_value(i, col)
                        if isinstance(valor, (int, float)) and 0 <= valor <= 100:
                            perc_criancas = valor
                            dados[capital] = perc_criancas
                            break
                    except:
                        continue
    
    return dados

Extraindo

In [44]:
dados_tabela2 = extrair_tabela2()
dados_tabela4 = extrair_tabela4()
dados_tabela6 = extrair_tabela6()
dados_tabela8 = extrair_tabela8()
dados_tabela19 = extrair_tabela19()